# Project 2 - Analyzing the NYC Subway Dataset
'''
Created by Laura Uzcategui 
Date : 30 Sept 2015 
'''

# Subway Ridership 

Subway ridership consists of all passengers (other than NYC Transit employees) who enter the subway system, including passengers who transfer from buses.  Ridership does not include passengers who exit the subway or passengers who transfer from other subway lines, with the exception of out-of-system transfers; e.g., between the Lexington Av/63 St F subwaystation and the Lexington Avenue N subwayQ subwayR subway/59 St4 subway5 subway6 subwaystation, where customers use their MetroCard to make the transfer.

This project has the purpose to analyze and investigate if more people ride the Subway when is raining vs when it is not raining. [1]

In [7]:
# making available graphs inline
%pylab inline
# importing pandas for manipulation of data
import pandas as pd

# Loading the path of our NYC subway dataset
filename = './turnstile_weather_v2.csv'

# Let's read our dataset
weather_data = pd.read_csv(filename)
#weather_data

Populating the interactive namespace from numpy and matplotlib


##Section 1. Statistical Test 

The appropiate hypotheses for this task will be the following: 
* H0 ( Null hypotheses) : $\mu C = \mu I $ 
The mean population from the Congruent Words are not different from the Mean the Incongruent Words

* H1 ( Alternative hypotheses) : $\mu C - \mu I  < 0 $ 
The difference of Congruent and Incongruent is less than zero ( One Tail Test) 

The statistical test to perfom will be : 2-Sample T-Test, since we have 2 samples with same size and We want to measure if the difference from the means are less than zero, so that this test will be a One Tail test. 

1.1 Which statistical test did you use to Analyze the NYC Subway data?

Did you use a one-tail or a two-tail P value? 

What is the null Hypothesis ? 

What is your p-critical value? 

In [2]:
import pandas as pd